In [ ]:
import pygrib
import os
import csv

def calc_pod(thr, dir_path, param_name, output_csv):

    THR = thr  # 閾値
    POD = [0]  # 閾値を超えるデータ(POD[-1]を使用するために初期値を設定)
    POD_IDX = [-168]  # 閾値を超えるデータのindex(POD_IDX[-1]を使用するために初期値を設定)
    CNT = 0  # indexカウント用の変数

    DIR_PATH = dir_path
    DIR = os.listdir(DIR_PATH)

    for filename in DIR:
        print(f'{filename}...now')
        grbs = pygrib.open(DIR_PATH + filename)
        grbs = grbs.select(parameterName=param_name)
        for grb in grbs:
            CNT += 1
            # まずは1地点でやるために, (20, 20)で試す
            # maskされた部分は0で埋める
            data = grb.data()[0].filled(fill_value=0)[20][20]
            # decluster
            if data > THR:
                if CNT > POD_IDX[-1] + 168:
                    POD.append(data)
                    POD_IDX.append(CNT)
                else:
                    if data > POD[-1]:
                        POD[-1] = data
                        POD_IDX[-1] = CNT

    # 初期値を削除
    POD.pop(0)
    POD_IDX.pop(0)

    # 書き出し
    f = open(output_csv, 'w')
    writer = csv.writer(f)
    writer.writerow(POD)
    writer.writerow(POD_IDX)
    f.close()

    return POD, POD_IDX, CNT, len(POD)

In [ ]:
from csv import reader
import matplotlib.pyplot as plt

def plot(filename, output_name):
    with open(filename, 'r') as csv_file:
        csv_reader = reader(csv_file)
        l = list(csv_reader)
        x = []
        y = []
        for i in range(len(l[0])):
            x.append(l[1][i])
            y.append(float(l[0][i]))

    fig = plt.figure()
    plt.plot(x, y, 'o')
    plt.xticks([])
    plt.xlabel("time")
    plt.ylabel("Hs[m]")
    plt.title("Peak Over Threshold of (20, 20)")
    fig.savefig(output_name)
    plt.show()

    return